*Importing Libraries*

In [1]:
import numpy as np
import pandas as pd

*Reading Data*

In [2]:
df = pd.read_csv('Invideo_assignment.csv')

In [3]:
df.head()

,ID,Created At,Video creation date
0,467537,2020-08-02,2020-08-23T18:36:12
1,467537,2020-08-02,2020-08-23T18:50:10
2,467537,2020-08-02,2020-08-23T19:21:07
3,467537,2020-08-02,2020-09-07T19:28:12
4,467537,2020-08-02,2020-09-07T19:37:42


*Checking Null values*

In [4]:
df.isnull().sum()

ID                          0
Created At                  0
Video creation date    372494
dtype: int64

*Converting The Created At and Video Creation date into day of the year*

In [5]:
df['Created_day']=pd.to_datetime(df['Created At'],format='%Y-%m-%d').dt.dayofyear
df['Vid_creation_day']=pd.to_datetime(df['Video creation date'],format='%Y-%m-%d').dt.dayofyear

In [6]:
df.head()

,ID,Created At,Video creation date,Created_day,Vid_creation_day
0,467537,2020-08-02,2020-08-23T18:36:12,215,236.0
1,467537,2020-08-02,2020-08-23T18:50:10,215,236.0
2,467537,2020-08-02,2020-08-23T19:21:07,215,236.0
3,467537,2020-08-02,2020-09-07T19:28:12,215,251.0
4,467537,2020-08-02,2020-09-07T19:37:42,215,251.0


In [7]:
pd.to_datetime('17/09/2020',format='%d/%m/%Y').dayofyear

261

*Removing the data which are after 17th sept. 2020, since the objective is to predict whether the user will create video after 3 months or not.*

In [8]:
df=df[df['Created_day']<=261]
df.shape

(284158, 5)

In [9]:
df.drop(['Created At','Video creation date'],axis=1,inplace=True)

*Checking the difference between the sign up day and video created day*

In [10]:
arr1=df['Vid_creation_day'].to_numpy()
arr2=df['Created_day'].to_numpy()
which_day=arr1-arr2
df['which_day']=which_day

In [13]:
df

,ID,Created_day,Vid_creation_day,which_day
0,467537,215,236.0,21.0
1,467537,215,236.0,21.0
2,467537,215,236.0,21.0
3,467537,215,251.0,36.0
4,467537,215,251.0,36.0
...,...,...,...,...
284153,633219,261,262.0,1.0
284154,633220,261,NaN,NaN
284155,633221,261,NaN,NaN
284156,633222,261,NaN,NaN


*Making a new dataframe which contains a single row for each user id and for each user there are 36 columns. 35 columns for day 1 to day 35 and 36th column is the output column which denotes the day 90+.*

In [14]:
z1=df.groupby('ID')['which_day'].apply(list)
dfnew=[]
for k,v in z1.iteritems():
    data=[0]*37
    if v[0]==-1:
            dfnew.append(data)
    else:
        for i in v:
            if i<=35:
                data[int(i)]+=1
            elif i>=90:
                data[-1]=1
        dfnew.append(data)

In [15]:
i=0
column_name=[]
while i<36:
    column_name.append('day'+str(i))
    i=i+1
df2=pd.DataFrame(dfnew,columns=column_name+['Output'])
df2

,day0,day1,day2,day3,day4,day5,day6,day7,day8,day9,...,day27,day28,day29,day30,day31,day32,day33,day34,day35,Output
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163924,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
163925,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
163926,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
163927,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


*Checking whether the data is balanced or imbalanced*

In [16]:
df2['Output'].value_counts()

0    162415
1      1514
Name: Output, dtype: int64

*Defining Dependent and Independent varaiable*

In [17]:
X=df2.iloc[:,0:36]
Y=df2.iloc[:,-1]

*Upsampling the data.* 

In [18]:
from imblearn.over_sampling import RandomOverSampler
os =  RandomOverSampler(sampling_strategy=.5)
X_res, y_res = os.fit_resample(X, Y)

In [19]:
from collections import Counter
print('Original dataset shape {}'.format(Counter(Y)))
print('Resampled dataset shape {}'.format(Counter(y_res)))

Original dataset shape Counter({0: 162415, 1: 1514})
Resampled dataset shape Counter({0: 162415, 1: 81207})


*Modelling The Data*

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_res,y_res,test_size=0.3, random_state=1)

In [21]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred=gnb.predict(X_test)

In [22]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.8360036668627799
[[47875   954]
 [11032 13226]]
              precision    recall  f1-score   support

           0       0.81      0.98      0.89     48829
           1       0.93      0.55      0.69     24258

    accuracy                           0.84     73087
   macro avg       0.87      0.76      0.79     73087
weighted avg       0.85      0.84      0.82     73087



In [23]:
ytrain_pred = gnb.predict_proba(X_train)

In [24]:
pred=gnb.predict_proba(X_test)[:,1]

In [25]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, pred)

In [26]:
from sklearn.metrics import f1_score
f1score = []
for thres in thresholds:
    y_pred = np.where(pred>thres,1,0)
    f1score.append(f1_score(y_test, y_pred))
    
f1score = pd.concat([pd.Series(thresholds), pd.Series(f1score)],axis=1)
f1score.columns = ['thresholds', 'f1_score']
f1score.sort_values(by='f1_score', ascending=False, inplace=True)
f1score.head()

,thresholds,f1_score
536,2.830022e-30,0.772756
537,2.775722e-30,0.772739
531,6.532000e-30,0.772687
541,8.219565e-31,0.772680
542,7.272214e-31,0.772645


In [27]:
new_pred=np.where(pred>f1score.iloc[0,0],1,0)

In [28]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print(accuracy_score(y_test,new_pred))
print(confusion_matrix(y_test,new_pred))
print(classification_report(y_test,new_pred))

0.8632314912364716
[[46095  2734]
 [ 7262 16996]]
              precision    recall  f1-score   support

           0       0.86      0.94      0.90     48829
           1       0.86      0.70      0.77     24258

    accuracy                           0.86     73087
   macro avg       0.86      0.82      0.84     73087
weighted avg       0.86      0.86      0.86     73087

